<a href="https://colab.research.google.com/github/lailimeifa/Laili_Meifa/blob/main/machine%20learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv('/content/drive/MyDrive/PPN12_SENTIMEN(2).csv')
df

,full_text,Sentimen
0,Masih dibahas aja mending bahas PPN 12% atau g...,Netral
1,Mau bangun pagi atau siang rejeki tetap dipato...,Negatif
2,@Dee_Rose82 Rakyat Jelata yg bakal d tarik ppn...,Negatif
3,Ini Jenis Mobil yang Bakal Kena PPN 12% ://t.c...,Netral
4,@BudiBukanIntel Yg terkait nasib hajat hidup o...,Positif
...,...,...
241,saat pemilu org kecil suaranya diperlukan sete...,Negatif
242,iya ih gue awalnya rispek sama dia krn memeran...,Negatif
243,@DedynurPalakka Tapi balas dendam nya lewat ke...,Negatif
244,Abang Jepang liat deh...kalo ppn naik 12% maki...,Negatif


In [3]:
def filter_tokens_by_length(dataframe, column, min_words, max_words):
    words_count = dataframe[column].astype(str).apply(lambda x: len(x.split()))
    mask = (words_count >= min_words) & (words_count <= max_words)
    filtered_df = dataframe[mask]
    return filtered_df

min_words = 3
max_words = 50
df = filter_tokens_by_length(df, 'full_text', min_words, max_words)

In [4]:
norm = {
    'yg': 'yang', 'udh': 'sudah', 'ga': 'tidak', 'brg': 'barang'}

def normalisasi(str_text):
    """Normalizes the input text by replacing abbreviations with full words."""
    for key, value in norm.items():
        str_text = str_text.replace(key, value)
    return str_text

df = df.copy()
df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))

In [5]:
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
stop_words = StopWordRemoverFactory().get_stop_words()
more_stop_words = []
stop_words.extend(more_stop_words)
stop_words_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(stop_words_array)
def stopword(str_text):
    return stop_words_remover_new.remove(str_text)
df['full_text'] = df['full_text'].apply(lambda x: stopword(x))

df.sample(5)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.6 MB/s eta 0:00:00


,full_text,Sentimen
85,yang bikin ppn 12% siapa deh bajintidakn gue r...,Negatif
232,jadi binun... katanya PPn 12% utk barang mewah...,Netral
166,Ppn 12% n now this. Akun2 educated ramein aja ...,Netral
17,@abitidakilimuriaa miftah mundur. skrg ayo bah...,Positif
65,Pemerintah Indonesia siap naikkan tarif PPN 12...,Netral


In [6]:
import re

def clean_twitter_text(text):
    # Menghapus karakter @mentions, #hashtags, dan URL
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Menghapus @mentions
    text = re.sub(r'#\w+', '', text)  # Menghapus #hashtags
    text = re.sub(r'RT[\s]+', '', text)  # Menghapus retweet (RT)
    text = re.sub(r'https?://\S+', '', text)  # Menghapus URL
    text = re.sub(r'[^A-Za-z0-9 ]', '', text)  # Menghapus karakter selain huruf dan angka
    text = re.sub(r'\s+', ' ', text).strip()  # Menghapus spasi berlebih dan memotong teks
    return text

# Menerapkan fungsi pada kolom 'full_text'
df['full_text'] = df['full_text'].apply(clean_twitter_text)

In [7]:
df['full_text'] = df['full_text'].str.lower()
df

,full_text,Sentimen
0,masih dibahas aja mending bahas ppn 12 genosid...,Netral
1,mau bangun pagi siang rejeki tetap dipatok ppn 12,Negatif
2,rakyat jelata bakal d tarik ppn 12 hehehe,Negatif
3,ini jenis mobil bakal kena ppn 12 tco0c1fdtoj6p,Netral
4,yg terkait nasib hajat hidup org byk spt semba...,Positif
...,...,...
241,pemilu org kecil suaranya diperlukan pemilu di...,Negatif
242,iya ih gue awalnya rispek sama krn memerangi j...,Negatif
243,tapi balas dendam nya lewat kebijakan bobrok d...,Negatif
244,abang jepang liat dehkalo ppn naik 12 makin wo...,Negatif


In [8]:
tokenized = df['full_text'].apply(lambda x:x.split())
tokenized

,full_text
0,"[masih, dibahas, aja, mending, bahas, ppn, 12,..."
1,"[mau, bangun, pagi, siang, rejeki, tetap, dipa..."
2,"[rakyat, jelata, bakal, d, tarik, ppn, 12, heh..."
3,"[ini, jenis, mobil, bakal, kena, ppn, 12, tco0..."
4,"[yg, terkait, nasib, hajat, hidup, org, byk, s..."
...,...
241,"[pemilu, org, kecil, suaranya, diperlukan, pem..."
242,"[iya, ih, gue, awalnya, rispek, sama, krn, mem..."
243,"[tapi, balas, dendam, nya, lewat, kebijakan, b..."
244,"[abang, jepang, liat, dehkalo, ppn, naik, 12, ..."


In [9]:
!pip install tweet-preprocessor
!pip install textblob
!pip install wordcloud
!pip install nltk

In [10]:
import pandas as pd

# Misalkan file Anda bernama 'data_tweet.csv'
df = pd.read_csv('/content/drive/MyDrive/PPN12_SENTIMEN(2).csv')

# Pastikan kolom 'Comment' ada di dalam data
print(df.columns)

Index(['full_text', 'Sentimen'], dtype='object')


In [11]:
import random

# Inisialisasi set untuk masing-masing kategori
set_positif = []
set_negatif = []
set_netral = []

# Asumsikan dataset adalah list of tuples, di mana elemen pertama adalah teks dan elemen kedua adalah label
# Contoh dataset: dataset = [("teks1", "Positif"), ("teks2", "Negatif"), ("teks3", "Netral")]
for n in df:
    if n[1] == 'Positif':
        set_positif.append(n)
    elif n[1] == 'Negatif':
        set_negatif.append(n)
    else:  # Netral
        set_netral.append(n)

# Sampling acak dari setiap set dengan separuh data
set_positif = random.sample(set_positif, k=int(len(set_positif) / 2))
set_negatif = random.sample(set_negatif, k=int(len(set_negatif) / 2))
set_netral = random.sample(set_netral, k=int(len(set_netral) / 2))

# Gabungkan kembali set yang sudah disampling
train = set_positif + set_negatif + set_netral

# Inisialisasi list untuk data latih
train_set = []
for n in train:
    train_set.append(n)

# Output akhir
print("Data latih (train_set):")
print(train_set)


Data latih (train_set):
['full_text']


In [14]:
# Import pustaka yang diperlukan
!pip install train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Asumsi df sudah memiliki kolom 'Comment' dan 'Sentimen'
# Langkah ini mengonversi data menjadi tuple (teks, label)
train_set = list(zip(df['full_text'], df['Sentimen']))

# Pisahkan teks dan label
texts, labels = zip(*train_set)

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=10)

# Preprocessing teks menggunakan CountVectorizer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Membuat model Naive Bayes
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)

# Prediksi pada data baru
sample_texts = ["Luhut Cs Ungkap Tujuan Penetapan PPN 12% Buat Barang Mewah", "Beban pajak yang terlalu tinggi akibat PPN merugikan"]
sample_features = vectorizer.transform(sample_texts)  # Transformasi teks
predictions = clf.predict(sample_features)

y_pred = clf.predict(X_test_counts)
# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model: {accuracy * 100:.2f}%")

# Contoh klasifikasi teks baru
sample_texts = ["Penerapan PPN sangat baik", "PPN tidak membawa dampak positif bagi masyarakat"]
sample_features = vectorizer.transform(sample_texts)
sample_predictions = clf.predict(sample_features)

print("Hasil Prediksi:")
for text, sentiment in zip(sample_texts, sample_predictions):
    print(f"{text} => {sentiment}")

ERROR: Could not find a version that satisfies the requirement train_test_split (from versions: none)
ERROR: No matching distribution found for train_test_split
Akurasi Model: 72.00%
Hasil Prediksi:
Penerapan PPN sangat baik => Negatif
PPN tidak membawa dampak positif bagi masyarakat => Positif
